# UNIT tests of temperament

In [6]:
import sys
sys.path.append("../../")

In [7]:
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import json
import numpy as np
import random
from collections import defaultdict, Counter
from analytics_lib.nlp_texts.psychotype import *

In [8]:
public_df = pd.read_pickle("../data/public_df.pkl")
public_semantic_role_df = pd.read_pickle("../data/public_semantic_role_df.pkl")
morph_df = pd.read_pickle("../data/morph_df.pkl")
    
df_with_ranges = df_with_sample_ranges_and_temp(public_df, public_semantic_role_df)

In [9]:
df_with_ranges

,Название характеристики,Меланхолик,Холерик,Флегматик,Сангвиник
0,Общее количество слов в тексте,-,-,-,-
1,Доля предложений на одно слово,"Меньше среднего: (0.06, 0.08)","Больше среднего: (0.1, 0.12)","Мало: (0.02, 0.06)","Много: (0.12, 0.25)"
2,Доля коротких предложений,"Среднее: (0.33, 0.5)","Среднее: (0.33, 0.5)","Много: (0.75, 1.57)","Мало: (0.0, 0.14)"
3,Доля длинных предложений,"Мало: (0.0, 0.25)","Больше среднего: (0.57, 0.75)","Мало: (0.0, 0.25)","Много: (0.75, 1.0)"
4,Доля длинных низкочастотных слов,"Среднее: (0.11, 0.13)","Мало: (0.0, 0.08)","Мало: (0.0, 0.08)","Много: (0.18, 0.28)"
5,Доля коротких высокочастотных слов,-,"Много: (0.58, 1.0)","Среднее: (0.48, 0.53)",-
6,Средняя длина предложения,-,-,-,-
7,Среднее число запятых в предложении,-,"Больше среднего: (1.25, 1.82)","Больше среднего: (1.25, 1.82)",-
8,Доля сложноподчиненных предложений,"Среднее: (0.17, 0.29)","Мало: (0.0, 0.0)","Мало: (0.0, 0.0)","Много: (0.5, 1.0)"
9,Доля восклицательных предложений,"Мало: (0.0, 0.0)","Много: (0.0, 0.5)",-,"Среднее: (0.0, 0.0)"


In [10]:
def processed_dict_with_features(dict_with_all_features, df_with_ranges):
    temp_dict_with_features = {"Морфология": {}, "Агенсность, Предикаты": {}}
    list_of_morph = list(df_with_ranges.index.values)
    list_of_semantic_role = ['Агенс',
                 'Пациенс',
                 'Внутренний предикат',
                 'Внешний предикат']
    for feature_name in list_of_morph:
        temp_dict_with_features["Морфология"][feature_name] =  dict_with_all_features.get(feature_name, -1)
    for feature_name in list_of_semantic_role:
        temp_dict_with_features["Агенсность, Предикаты"][feature_name] =  dict_with_all_features.get(feature_name, -1)
        
    res_dict_with_features = {}
    res_dict_with_features["text_feat"] = temp_dict_with_features
    
    return res_dict_with_features

In [22]:
def dict_with_features_by_temp(psychotype, df_with_ranges):
    dict_with_features = {}
    list_temp = ["Сангвиник", "Флегматик", "Меланхолик", "Холерик"]
    dict_with_psychotypes = dict(zip(["Сангвиник", "Флегматик", "Меланхолик", "Холерик"], [0] * len(list_temp)))
    df = df_with_ranges.set_index("Название характеристики")
    for feature_name in list(df.index.values):
        index_of_end = df.at[feature_name, psychotype].find(":")
        if index_of_end == -1:
            #dict_with_features[feature_name] = [-1, []]
            dict_with_features[feature_name] = -1
            continue
        range_ = df.at[feature_name, psychotype][ :index_of_end]
        tuple_range = eval(df.at[feature_name, psychotype][index_of_end + 2:])
        if abs(tuple_range[0] - tuple_range[1]) < 1e-10:
            continue
        # print("\n\n")
        # print(df.at[feature_name, psychotype])
        # print(tuple_range)
        #dict_with_local_psych = defaultdict(int)
        for key in list_temp:
            if size_by_feature_name_and_temp(df, feature_name, key) == range_:
                dict_with_psychotypes[f"{key}"] += 1
                #dict_with_local_psych[f"{key}"] += 1
                
        dict_with_features[feature_name] = random.uniform(tuple_range[0], tuple_range[1])
        #dict_with_features[feature_name] = [random.uniform(tuple_range[0], tuple_range[1]), [key for key in dict_with_psychotypes.keys() if dict_with_local_psych[key]]]
        #dict_with_local_psych = defaultdict(int)
        
    res_dict = processed_dict_with_features(dict_with_features, df)
    dict_with_psychotypes = {key: value for key, value in dict_with_psychotypes.items() if value}
    res_dict["psychotype"] = dict_with_psychotypes
    return res_dict

In [23]:
def size_by_feature_name_and_temp(df, feature_name, psychotype):
    index_of_end = df.at[feature_name, psychotype].find(":")
    if index_of_end == -1:
        size = "-"
        return size
    size = df.at[feature_name, psychotype][ :index_of_end]
    return size

## Сохранение в json по 5 тестов на каждый темперамент

In [24]:
%%time
for psych in ["Сангвиник", "Флегматик", "Меланхолик", "Холерик"]:
    for i in range(5):
        with open(f'tests/{psych}_{i}.json', 'w', encoding = "utf8") as f:
            json.dump(dict_with_features_by_temp(psych, df_with_ranges), f, ensure_ascii=False)

CPU times: user 55.5 ms, sys: 6.67 ms, total: 62.1 ms
Wall time: 62.2 ms


In [25]:
def df_from_dict(dict_input):
    dict_input['Текст'] = 'Проверочный текст'
    dict_res = {k[0]: [k[1]] for k in dict_input.items()}
    df = pd.DataFrame.from_dict(dict_res)
    return df

In [26]:
list_of_all_tests = []
list_of_psychotype = ["Сангвиник", "Флегматик", "Меланхолик", "Холерик"]
for psych in list_of_psychotype:
    for i in range(5):
        dict_temp = {}  # словарь для проверки
        with open(f'tests/{psych}_{i}.json', encoding='utf-8') as json_file:
            dict_temp = json.load(json_file)
            list_of_all_tests.append(dict_temp)

In [28]:
list_of_tests_res = []
for dict_test in list_of_all_tests:
    person_public_df = df_from_dict(dict_test['text_feat']['Морфология'])
    person_public_semantic_role_df = df_from_dict(dict_test['text_feat']['Агенсность, Предикаты'])
    text = 'Проверочный текст'
    dict_psych = dict(dict_with_temp_by_text(text, person_public_df, person_public_semantic_role_df, public_df, public_semantic_role_df))
    
    dict_test_psych = {k: v for k, v in dict_test["psychotype"].items()}
    assert dict_test_psych == dict_psych, "Несовпадение"